## Setting up model

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRFRegressor

import scipy
from sklearn.model_selection import train_test_split
import ipywidgets as widgets
from sklearn.preprocessing import StandardScaler

import pyarrow.feather as feather

import pickle

In [22]:
pd.set_option('display.max_columns', None)

## Submission Example
example of what final submission should look like

In [23]:
example_submit = pd.read_csv('data/example_sample_submission.csv')

In [24]:
example_submit.head()

,group_num,row_id,Target
0,0,0,0
1,0,1,0
2,0,2,0
3,0,3,0
4,0,4,0


## Test Example
example of what the test set will look like

In [25]:
example_test = pd.read_csv('data/example_test.csv')

In [26]:
example_test.head()

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,group_num,row_id
0,1623542400,3,1201.0,1.478556,1.48603,1.47800,1.483681,654799.561103,1.481439,0,0
1,1623542400,2,1020.0,580.306667,583.89000,579.91000,582.276667,1227.988328,581.697038,0,1
2,1623542400,0,626.0,343.789500,345.10800,343.64000,344.598000,1718.832569,344.441729,0,2
3,1623542400,1,2888.0,35554.289632,35652.46465,35502.67000,35602.004286,163.811537,35583.469303,0,3
4,1623542400,4,433.0,0.312167,0.31260,0.31192,0.312208,585577.410442,0.312154,0,4


## Coin Combo Dicitonary
dictionary to be used by models

In [27]:
coin_dict = {'Bitcoin Cash': 'DEFAULT',
             'Binance Coin': ['Bitcoin', 'EOS.IO', 'Dogecoin'],
             'Bitcoin': ['Dogecoin', 'Ethereum Classic', 'Cardano'],
             'EOS.IO': ['Bitcoin Cash', 'Bitcoin', 'Ethereum'],
             'Ethereum Classic': ['Bitcoin Cash'],
             'Ethereum': ['Stellar', 'Ethereum Classic'],
             'Litecoin': ['Dogecoin', 'Bitcoin Cash'],
             'Monero': ['TRON', 'Cardano'],
             'TRON': ['Stellar'],
             'Stellar': ['TRON'],
             'Cardano': ['IOTA', 'Bitcoin Cash', 'Monero'],
             'IOTA': 'DEFAULT',
             'Maker': ['Dogecoin', 'Stellar'],
             'Dogecoin': ['Ethereum Classic'],
            }

## Default Drops
features dropped for prediction

In [28]:
default_drops = ['Asset_ID', 'group_num', 'row_id']

## Load in asset details w coin names

In [29]:
asset_details = pd.read_csv('data/asset_details.csv')
asset_info = dict(zip(asset_details.Asset_ID, asset_details.Asset_Name))
asset_info_rev = dict(zip(asset_details.Asset_Name, asset_details.Asset_ID, ))

In [30]:
asset_details

,Asset_ID,Weight,Asset_Name
0,2,2.397895,Bitcoin Cash
1,0,4.304065,Binance Coin
2,1,6.779922,Bitcoin
3,5,1.386294,EOS.IO
4,7,2.079442,Ethereum Classic
5,6,5.894403,Ethereum
6,9,2.397895,Litecoin
7,11,1.609438,Monero
8,13,1.791759,TRON
9,12,2.079442,Stellar


## Rename Assets to coin names

In [31]:
example_test['Asset_ID'] = example_test['Asset_ID'].map(asset_info)

example_test.head()

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,group_num,row_id
0,1623542400,Cardano,1201.0,1.478556,1.48603,1.47800,1.483681,654799.561103,1.481439,0,0
1,1623542400,Bitcoin Cash,1020.0,580.306667,583.89000,579.91000,582.276667,1227.988328,581.697038,0,1
2,1623542400,Binance Coin,626.0,343.789500,345.10800,343.64000,344.598000,1718.832569,344.441729,0,2
3,1623542400,Bitcoin,2888.0,35554.289632,35652.46465,35502.67000,35602.004286,163.811537,35583.469303,0,3
4,1623542400,Dogecoin,433.0,0.312167,0.31260,0.31192,0.312208,585577.410442,0.312154,0,4


## Master Dataframe function
Makes a main dataframe for all models


In [81]:
def master_dataframe(df):
    #generate all timestamps
    all_time = df.drop_duplicates(subset=['timestamp'])[['timestamp']]

    #for each asset
    for i in list(asset_info.values()):
        #filter to just asset id values
        #df_1 = df[df['Asset_ID'] == i].drop(['Asset_ID'], axis=1)
        df_1 = df[df['Asset_ID'] == i].drop(default_drops, axis=1)
        
        #generate column dictionary
        column_dict = {}
        for i2 in df_1.drop('timestamp', axis=1).columns:
            column_dict[i2] = i2+"_{}".format(i)
        
        #rename columns to match asset_id
        df_1 = df_1.rename(columns=column_dict)
        
         #add to all time dictionary
        all_time = pd.merge(all_time, df_1 , how="outer", on='timestamp')
    all_time['Unnamed: 0'] = pd.to_datetime(all_time.timestamp, unit='s')
    all_time['week'] = all_time['Unnamed: 0'].dt.week
    all_time['year'] = all_time['Unnamed: 0'].dt.year
    all_time['month'] = all_time['Unnamed: 0'].dt.month

    all_time['minute'] = all_time['Unnamed: 0'].dt.minute
    all_time['dayofweek'] = all_time['Unnamed: 0'].dt.dayofweek
    all_time['day'] = all_time['Unnamed: 0'].dt.day
    all_time = all_time.drop('Unnamed: 0', axis=1)
    return all_time

## Make Master Dataframe

In [33]:
refined_dataset = master_dataframe(example_test)
refined_dataset.head()

,timestamp,Count_Bitcoin Cash,Open_Bitcoin Cash,High_Bitcoin Cash,Low_Bitcoin Cash,Close_Bitcoin Cash,Volume_Bitcoin Cash,VWAP_Bitcoin Cash,Count_Binance Coin,Open_Binance Coin,High_Binance Coin,Low_Binance Coin,Close_Binance Coin,Volume_Binance Coin,VWAP_Binance Coin,Count_Bitcoin,Open_Bitcoin,High_Bitcoin,Low_Bitcoin,Close_Bitcoin,Volume_Bitcoin,VWAP_Bitcoin,Count_EOS.IO,Open_EOS.IO,High_EOS.IO,Low_EOS.IO,Close_EOS.IO,Volume_EOS.IO,VWAP_EOS.IO,Count_Ethereum Classic,Open_Ethereum Classic,High_Ethereum Classic,Low_Ethereum Classic,Close_Ethereum Classic,Volume_Ethereum Classic,VWAP_Ethereum Classic,Count_Ethereum,Open_Ethereum,High_Ethereum,Low_Ethereum,Close_Ethereum,Volume_Ethereum,VWAP_Ethereum,Count_Litecoin,Open_Litecoin,High_Litecoin,Low_Litecoin,Close_Litecoin,Volume_Litecoin,VWAP_Litecoin,Count_Monero,Open_Monero,High_Monero,Low_Monero,Close_Monero,Volume_Monero,VWAP_Monero,Count_TRON,Open_TRON,High_TRON,Low_TRON,Close_TRON,Volume_TRON,VWAP_TRON,Count_Stellar,Open_Stellar,High_Stellar,Low_Stellar,Close_Stellar,Volume_Stellar,VWAP_Stellar,Count_Cardano,Open_Cardano,High_Cardano,Low_Cardano,Close_Cardano,Volume_Cardano,VWAP_Cardano,Count_IOTA,Open_IOTA,High_IOTA,Low_IOTA,Close_IOTA,Volume_IOTA,VWAP_IOTA,Count_Maker,Open_Maker,High_Maker,Low_Maker,Close_Maker,Volume_Maker,VWAP_Maker,Count_Dogecoin,Open_Dogecoin,High_Dogecoin,Low_Dogecoin,Close_Dogecoin,Volume_Dogecoin,VWAP_Dogecoin,week,year,month,minute,dayofweek,day
0,1623542400,1020.0,580.306667,583.89,579.91,582.276667,1227.988328,581.697038,626.0,343.7895,345.108,343.64,344.5980,1718.832569,344.441729,2888.0,35554.289632,35652.464650,35502.67,35602.004286,163.811537,35583.469303,359.0,4.832550,4.8459,4.8229,4.837583,47143.548386,4.836607,541.0,55.223080,55.4940,55.1820,55.34468,6625.201803,55.298816,2186.0,2371.194286,2379.200000,2369.67,2374.380714,1214.128692,2374.335307,560.0,161.933429,162.48,161.73,162.214714,1485.009496,162.231310,123.0,243.1375,243.81,242.96,243.5325,307.958853,243.452697,229.0,0.068132,0.068240,0.068038,0.068158,3.046438e+06,0.068158,383.0,0.327973,0.329272,0.327650,0.328829,536491.101687,0.328582,1201.0,1.478556,1.486030,1.478000,1.483681,654799.561103,1.481439,35.0,1.00315,1.0198,0.9873,1.00330,7061.927600,1.002936,61.0,2939.862750,2952.1600,2936.23,2947.078025,9.584785,2945.110614,433.0,0.312167,0.3126,0.31192,0.312208,5.855774e+05,0.312154,23,2021,6,0,6,13
1,1623542460,1251.0,581.800000,585.59,580.38,582.358333,1405.285079,583.451389,458.0,344.3535,344.790,343.62,344.0895,1217.352439,344.188716,2006.0,35596.771429,35621.000000,35533.38,35555.397143,93.363659,35584.861196,644.0,4.834550,4.8495,4.8185,4.838300,86783.074424,4.837430,530.0,55.320900,55.4266,55.1490,55.28640,6929.457933,55.265491,1261.0,2373.970101,2375.350000,2369.37,2371.790000,786.738453,2372.809830,381.0,162.141857,162.38,161.80,162.193000,1344.809595,162.121310,62.0,243.3680,243.50,242.94,243.3300,143.505320,243.337398,395.0,0.068144,0.068334,0.068040,0.068217,4.981365e+06,0.068201,287.0,0.328656,0.329280,0.327780,0.328563,449638.981788,0.328548,672.0,1.482410,1.483759,1.479200,1.482043,285828.559844,1.481495,98.0,1.00305,1.0196,0.9841,1.00155,47719.510738,1.001078,35.0,2946.370567,2949.7200,2943.71,2945.711900,2.794134,2946.633163,573.0,0.312274,0.3124,0.31154,0.311847,9.396433e+05,0.311930,23,2021,6,1,6,13
2,1623542520,540.0,582.100000,582.67,579.62,580.540000,472.373005,580.850017,535.0,343.7780,344.060,342.50,343.0095,975.797047,343.139652,3531.0,35550.271250,35576.590000,35402.87,35488.287500,220.535164,35480.068897,380.0,4.833440,4.8357,4.8143,4.822080,41288.479550,4.824327,825.0,55.225700,55.3096,54.9100,55.02605,12315.492361,55.095780,1856.0,2370.880011,2371.950076,2363.00,2365.590000,764.080469,2367.128372,439.0,161.876143,162.03,161.46,161.829143,1265.968720,161.737656,49.0,242.7450,243.00,242.23,242.3475,54.208340,242.451070,338.0,0.068159,0.068170,0.067950,0.068025,2.790134e+06,0.068057,264.0,0.328061,0.328370,0.326973,0.327497,742486.816952,0.327676,849.0,1.

## Custom Dataframe Function
for each coin, make a cust dataframe using coin_dict

In [65]:
def custom_dataframe(df, coin, param_list):
    #print((coin, param_list))
    custom_dataset = df
    #For each coin
    for c in param_list:
        #DROP COIN
        dropped_coin = [k for k in df.columns if c in k]
        if (c == 'Bitcoin') | (c == 'Ethereum'):
            dropped_coin = [k for k in dropped_coin if ' ' not in k]
        custom_dataset = custom_dataset.drop(dropped_coin, axis=1)   
    return custom_dataset

## Make Predictions for all models
For each coin, load in a  scalar and model then use them to make a prediction

In [66]:
%%time

#stores_datasets with predictions & timestamps
predict_dict = {}

#for each coin
for i in asset_details.Asset_Name:

    
    ######MAYBE DROP AT SOME POINT???
    if coin_dict[i] == "DEFAULT":
        #use default dataframe
        dataframe_coin = refined_dataset.copy()
    else:
        #generate custom dataframe
        dataframe_coin = custom_dataframe(refined_dataset.copy(), i, coin_dict[i])
    ######MAYBE DROP AT SOME POINT???
    
    
    #load in scalar    
    sc = pickle.load(open('scalers/scaler_{}.pkl'.format(i),'rb'))
    
    X_values = dataframe_coin
    X_values.replace([np.inf, -np.inf], 0, inplace=True)
    #fill any missing values with median
    
    #load in model
    model = XGBRFRegressor()

    model.load_model("models/model_{}.json".format(i))
    
    #scale the data
    x_testScaled = sc.transform(X_values)
    
    #make a prediction
    y_pred = model.predict(x_testScaled)

    
    #make predictions
    X_values['Target'] = y_pred.tolist() #added in to list
    
    #set the asset it to the correct coin
    X_values['Asset_ID'] = i
    
    #add to the dictionary
    predict_dict[i] = X_values[['timestamp', 'Asset_ID', 'Target']]
    
    
    #delete the model and scaler
    del model
    del sc
    del dataframe_coin
    del x_testScaled
    del X_values

CPU times: user 12min 51s, sys: 6min 1s, total: 18min 53s
Wall time: 21min 58s


#  Matches coin to time stamp and save submission

In [69]:

def target_finder(time, coin):
    x_1 = predict_dict[coin]
    return float(x_1[(x_1['timestamp']==time) & (x_1['Asset_ID']==coin)].Target)


#
r = example_test
r['Target'] = r.apply(lambda x: target_finder(x.timestamp, x.Asset_ID), axis=1)
submission = r[['group_num', 'row_id', 'Target']]
submission.to_csv('submission.csv')

In [76]:
submission.head()

,group_num,row_id,Target
0,0,0,0.499833
1,0,1,0.499833
2,0,2,0.499833
3,0,3,0.499833
4,0,4,0.499833


In [77]:
example_submit.head()

,group_num,row_id,Target
0,0,0,0
1,0,1,0
2,0,2,0
3,0,3,0
4,0,4,0
